In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/MyDrive/cough_01/Test Code 1

/content/gdrive/MyDrive/cough_01/Test Code 1


In [1]:
import sounddevice as sd
from scipy.io.wavfile import write

In [2]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm ?

ModuleNotFoundError: No module named 'google.colab'

In [3]:
import librosa as lb
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
audio = record()

NameError: name 'record' is not defined

In [18]:
sound, sample_rate = lb.load(audio)
stft = np.abs(lb.stft(sound))  
mfccs = np.mean(lb.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
chroma = np.mean(lb.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
mel = np.mean(lb.feature.melspectrogram(sound, sr=sample_rate),axis=1)
contrast = np.mean(lb.feature.spectral_contrast(S=stft, sr=sample_rate),axis=1)
tonnetz = np.mean(lb.feature.tonnetz(y=lb.effects.harmonic(sound), sr=sample_rate),axis=1)
    
concat = np.concatenate((mfccs,chroma,mel,contrast,tonnetz))

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=1013
  n_fft, y.shape[-1]


In [19]:
model = keras.models.load_model("exp-2_1.h5")

In [20]:
images_tes = np.array(concat)

In [21]:
images_tes = np.reshape(images_tes, (1, images_tes.shape[0], 1))
print(images_tes.shape)

(1, 193, 1)


In [22]:
preds = model.predict(images_tes)

In [23]:
classpreds = np.argmax(preds, axis=1) # predicted classes 

In [24]:
print(classpreds)

[1]


In [25]:
 if (classpreds == 0):
        print("positive")
elif (classpreds== 1):
        print("negative")

talk
